# RPLib Problem 001
## NCAA Men's Basketball Dataset

In [1]:
major_description = "First representative example for the rankability library. Built around the study of NCAA Men's Basketball league."
print(major_description)

First representative example for the rankability library. Built around the study of NCAA Men's Basketball league.


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [26]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import itertools
import joblib
from scipy import stats

In [4]:
from pathlib import Path
home = str(Path.home())
home

'/home/jupyter-pander14'

In [5]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

In [6]:
sys.path.insert(0,"%s/marchmadness_study/"%home)
import base

In [7]:
sys.path.insert(0,"%s/sensitivity_study/src"%home)
import sensitivity_tests
import utilities

In [8]:
games={}
remaining_games={}
madness_teams={}
all_teams={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year],remaining_games[year] = base.read_data(f'{home}/marchmadness_study/data/%steams.txt'%year,f'{home}/marchmadness_study/data/%sgames.txt'%year,f'{home}/marchmadness_study/data/%sMadnessTeams.txt'%year)
    madness_teams[year] = list(np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1])))
    all_teams[year] = list(np.unique(list(games[year].team1_name) + list(games[year].team2_name)))
print(year)
games[year]

2018


,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
4147,256,170,737008,2017-11-10,1,92,-1,77,Minnesota,SC_Upstate,0,0
4288,265,293,737008,2017-11-10,1,75,-1,50,Texas_Tech,South_Alabama,1,0
925,56,326,737008,2017-11-10,1,75,-1,60,Villanova,Columbia,1,0
521,32,176,737008,2017-11-10,1,79,-1,78,Monmouth_NJ,Bucknell,0,1
4252,263,235,737008,2017-11-10,1,105,-1,74,Purdue,SIUE,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1985,121,152,737122,2018-03-04,0,65,0,49,Loyola-Chicago,Illinois_St,1,0
933,57,114,737122,2018-03-04,1,81,-1,71,Houston,Connecticut,1,0
1370,81,164,737122,2018-03-04,1,90,-1,70,Memphis,East_Carolina,0,0
1483,88,143,737122,2018-03-04,-1,108,1,96,Lipscomb,FL_Gulf_Coast,1,0


## What does a dataset look like?

In [9]:
remaining_games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2,1,73,737130,2018-03-12,1,80,-1,73,Drake,Abilene_Chr,0,0
32,2,313,737125,2018-03-07,1,97,-1,90,UNLV,Air_Force,0,0
42,3,79,737126,2018-03-08,0,67,0,58,E_Michigan,Akron,0,0
59,4,137,737128,2018-03-10,0,86,0,63,Kentucky,Alabama,1,1
68,4,326,737135,2018-03-17,0,81,0,58,Villanova,Alabama,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5478,347,287,737133,2018-03-15,0,73,0,47,Tennessee,Wright_St,1,1
5491,348,197,737126,2018-03-08,0,85,0,75,New_Mexico,Wyoming,0,0
5496,349,92,737136,2018-03-18,0,75,0,70,Florida_St,Xavier,1,1
5498,349,234,737127,2018-03-09,0,75,0,72,Providence,Xavier,1,1


## Running Massey and Colley
Parameters are selected below

In [10]:
direct_thress = [0]
spread_thress = [0]
weight_indirects = [0]
domains_ranges = [('all','madness')]

Fractions represent points in the session to create a dataset and run the algorithms

In [11]:
games['2002']

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2237,151,10,731162,2001-11-08,0,71,0,67,Arizona,Maryland,1,1
3937,264,86,731162,2001-11-08,0,72,0,64,Florida,Temple,1,0
1285,86,10,731163,2001-11-09,0,75,0,71,Arizona,Florida,1,1
3940,264,151,731163,2001-11-09,0,82,0,74,Maryland,Temple,1,0
2204,147,261,731166,2001-11-12,1,78,-1,58,Syracuse,Manhattan,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1950,133,8,731277,2002-03-03,0,66,0,58,American_Univ,Lafayette,0,0
2939,198,123,731277,2002-03-03,0,90,0,84,IUPUI,Oakland,0,0
2410,164,128,731277,2002-03-03,-1,95,1,92,Kansas,Missouri,1,1
3704,249,165,731277,2002-03-03,0,70,0,57,Missouri_KC,Southern_Utah,0,0


In [12]:
from datetime import timedelta

days_to_subtracts = [int(d) for d in np.arange(7+28,0,-7)]#[7+28]

delta = timedelta(days=days_to_subtracts[0]) # sample
delta

datetime.timedelta(days=35)

In [13]:
days_to_subtracts

[35, 28, 21, 14, 7]

In [14]:
massey_rankings = {}
colley_rankings = {}
massey_rs = {}
colley_rs = {}
colley_perms = {}
massey_perms = {}

outer_keys = list(itertools.product(domains_ranges,years))
for domain_range,year in tqdm(outer_keys):
    # set the team_domain
    team_domain = None
    if domain_range[0] == 'madness':
        team_domain = madness_teams[year]
    elif domain_range[0] == 'all':
        team_domain = all_teams[year]

    # set the team_range
    team_range = None
    if domain_range[1] == 'madness':
        team_range = madness_teams[year]
    elif domain_range[1] == 'all':
        team_range = all_teams[year]

    columns = ["days_to_subtract","direct_thres","spread_thres","weight_indirect"]+team_range
    massey_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rankings[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_rs[(domain_range,year)] = pd.DataFrame(columns=columns)
    massey_perms[(domain_range,year)] = pd.DataFrame(columns=columns)
    colley_perms[(domain_range,year)] = pd.DataFrame(columns=columns)

    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date')#.drop('date',axis=1)
    mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
    game_df = game_df.loc[mask]

    keys = list(itertools.product(days_to_subtracts,direct_thress,spread_thress,weight_indirects))

    def compute(days_to_subtract,direct_thres,spread_thres,weight_indirect,team_range,all_teams,game_df,selection_sunday):
        #upper = int(len(game_df)*frac)
        #game_df_sample = game_df.iloc[:upper,:]
        delta = timedelta(days=days_to_subtract)
        game_df_sample = game_df.loc[game_df["date"] <= pd.to_datetime(selection_sunday,format="%m/%d/%Y")-delta].drop('date',axis=1)

        map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres)
        colley_matrix,colley_b,indirect_colley_matrix,indirect_colley_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        colley_matrix = colley_matrix.reindex(index=all_teams,columns=all_teams)
        colley_b = colley_b.reindex(all_teams)
        indirect_colley_matrix = indirect_colley_matrix.reindex(index=all_teams,columns=all_teams)
        indirect_colley_b = indirect_colley_b.reindex(all_teams)
        inxs = []
        for team in team_range:
            inxs.append(int(np.where(colley_b.index == team)[0][0]))
        ranking1,r1,perm1 = pyrankability.rank.ranking_from_matrices(colley_matrix.fillna(0),colley_b.fillna(0),np.array(inxs))
        indirect_ranking1,indirect_r1,indirect_perm1 = pyrankability.rank.ranking_from_matrices(indirect_colley_matrix.fillna(0),indirect_colley_b.fillna(0),np.array(inxs))
        if weight_indirect > 0:
            r1 = r1+weight_indirect*indirect_r1
            perm1, ranking1 = pyrankability.rank.perm_ranking_from_r(r1)
        
        map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=direct_thres,spread_thres=spread_thres)
        massey_matrix,massey_b,indirect_massey_matrix,indirect_massey_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        massey_matrix = massey_matrix.reindex(index=all_teams,columns=all_teams)
        massey_b = massey_b.reindex(all_teams)
        indirect_massey_matrix = indirect_massey_matrix.reindex(index=all_teams,columns=all_teams)
        indirect_massey_b = indirect_massey_b.reindex(all_teams)
        inxs = []
        for team in team_range:
            inxs.append(int(np.where(massey_b.index == team)[0][0]))
        ranking2,r2,perm2 = pyrankability.rank.ranking_from_matrices(massey_matrix.fillna(0),massey_b.fillna(0),np.array(inxs))
        indirect_ranking2,indirect_r2,indirect_perm2 = pyrankability.rank.ranking_from_matrices(massey_matrix.fillna(0),massey_b.fillna(0),np.array(inxs))
        if weight_indirect > 0:
            r2 = r2+weight_indirect*indirect_r2
            perm2, ranking2 = pyrankability.rank.perm_ranking_from_r(r2)
            
        ranking_values1 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(ranking1)
        ranking_values2 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(ranking2)
        r_values1 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(r1)
        r_values2 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(r2)
        perm_values1 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(perm1)
        perm_values2 = [days_to_subtract,direct_thres,spread_thres,weight_indirect]+list(perm2)
        
        return (pd.Series(ranking_values1,index=columns),
                pd.Series(ranking_values2,index=columns),
                pd.Series(r_values1,index=columns),
                pd.Series(r_values2,index=columns),
                pd.Series(perm_values1,index=columns),
                pd.Series(perm_values2,index=columns)              
               )

    #frac,direct_thres,spread_thres,weight_indirect = keys[0]
    #for frac,direct_thres,spread_thres,weight_indirect in keys:
    #    compute(frac,direct_thres,spread_thres,weight_indirect,team_range,all_teams[year],game_df)
    results = Parallel(n_jobs=-1)(delayed(compute)(days_to_subtract,direct_thres,spread_thres,weight_indirect,team_range,all_teams[year],game_df,base.selectionSundays[year]) for days_to_subtract,direct_thres,spread_thres,weight_indirect in keys)

    c = 0
    for i,key in enumerate(keys):
        days_to_subtract,direct_thres,spread_thres,weight_indirect = key
        colley,massey,colley_r,massey_r,colley_perm,massey_perm = results[i]
        massey.name = c
        colley.name = c
        colley_r.name=c
        massey_r.name=c
        colley_perm.name=c
        massey_perm.name=c
        massey_rankings[(domain_range,year)] = massey_rankings[(domain_range,year)].append(massey)
        colley_rankings[(domain_range,year)] = colley_rankings[(domain_range,year)].append(colley)
        massey_rs[(domain_range,year)] = massey_rs[(domain_range,year)].append(massey_r)
        colley_rs[(domain_range,year)] = colley_rs[(domain_range,year)].append(colley_r)
        massey_perms[(domain_range,year)] = massey_perms[(domain_range,year)].append(massey_perm)
        colley_perms[(domain_range,year)] = colley_perms[(domain_range,year)].append(colley_perm)
        c+=1

100%|██████████| 17/17 [01:31<00:00,  5.77s/it]


## What do we have after running

In [15]:
display(colley_rankings[(domain_range,year)])

,days_to_subtract,direct_thres,spread_thres,weight_indirect,Alabama,Arizona,Arkansas,Auburn,Bucknell,Buffalo,...,Texas_Tech,UMBC,UNC_Greensboro,Villanova,Virginia,Virginia_Tech,West_Virginia,Wichita_St,Wright_St,Xavier
0,35,0,0,0,32,16,34,5,54,41,...,13,63,50,2,1,44,23,22,51,4
1,28,0,0,0,34,16,27,4,58,44,...,11,62,50,2,1,37,22,17,52,3
2,21,0,0,0,31,16,21,5,56,45,...,10,61,49,2,1,38,23,13,54,3
3,14,0,0,0,40,17,24,7,55,46,...,19,63,49,3,1,37,22,12,56,2
4,7,0,0,0,47,14,24,9,53,44,...,19,63,48,3,1,35,23,12,55,2


## What parameters should we pick?
One way to focus our analysis is to study the parameters that result in the best accuracy when predicting future games. This code isn't really necessary if you picked a single set of parameters. We used only regular season games to create our ranking (cut off at different points in the season). Now we can evaluate on the remaining games.

In [16]:
def calc_predictability(games,r):
    numberCorrectPredictions = 0
    numGames = 0
    for i in games.index:
        team1ID = games.loc[i, "team1_name"]
        team1Score = games.loc[i, "points1"]
        team2ID = games.loc[i, "team2_name"]
        team2Score = games.loc[i, "points2"]
        
        if team1ID in r.index and team2ID in r.index:
            if team1Score > team2Score and r.loc[team1ID] > r.loc[team2ID]:
                numberCorrectPredictions += 1
            elif team2Score > team1Score and r.loc[team2ID] > r.loc[team1ID]:
                numberCorrectPredictions += 1
            elif team1Score == team2Score and r.loc[team1ID] == r.loc[team2ID]:
                numberCorrectPredictions += 1

            numGames += 1
           
    predictability = numberCorrectPredictions/numGames*100
    return predictability,numGames

In [17]:
pred_df = pd.DataFrame(columns=['days_to_subtract','domain','range',"direct_thres","spread_thres","weight_indirect",'Method','Year','Predictability','rankings'])

keys = list(itertools.product(days_to_subtracts,domains_ranges,direct_thress,spread_thress,weight_indirects,years))

c=0
for days_to_subtract,domain_range,dt,st,iw,year in tqdm(keys):
    dom = domain_range[0]
    ran = domain_range[1]
    method = 'Massey'
    r = massey_rs[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    rankings = massey_rankings[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    val = calc_predictability(remaining_games[year],r)[0]
    entry = pd.Series([days_to_subtract,dom,ran,dt,st,iw,method,year,val,rankings],name=c,index=pred_df.columns)
    c+=1
    pred_df=pred_df.append(entry)

    method = 'Colley'
    r = colley_rs[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    rankings = colley_rankings[(domain_range,year)].set_index(["days_to_subtract","direct_thres","spread_thres","weight_indirect"]).loc[(days_to_subtract,dt,st,iw)]
    val = calc_predictability(remaining_games[year],r)[0]
    entry = pd.Series([days_to_subtract,dom,ran,dt,st,iw,method,year,val,rankings],name=c,index=pred_df.columns)
    c+=1
    pred_df=pred_df.append(entry)
    

100%|██████████| 85/85 [00:02<00:00, 31.64it/s]


In [18]:
pred_df

,days_to_subtract,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,Predictability,rankings
0,35,all,madness,0,0,0,Massey,2002,69.662921,Alabama 10 Alcorn_St 65 Ariz...
1,35,all,madness,0,0,0,Colley,2002,65.168539,Alabama 6 Alcorn_St 62 Ariz...
2,35,all,madness,0,0,0,Massey,2003,65.476190,Alabama 36 Arizona 3 Arizona_St...
3,35,all,madness,0,0,0,Colley,2003,64.285714,Alabama 33 Arizona 2 Arizona_St...
4,35,all,madness,0,0,0,Massey,2004,63.440860,Air_Force 39 Alabama 30 Alabama_St...
...,...,...,...,...,...,...,...,...,...,...
165,7,all,madness,0,0,0,Colley,2016,71.428571,Akron 51 Arizona 16 Ar...
166,7,all,madness,0,0,0,Massey,2017,54.838710,Arizona 26 Arkansas 35 Baylor...
167,7,all,madness,0,0,0,Colley,2017,54.838710,Arizona 6 Arkansas 27 Baylor...
168,7,all,madness,0,0,0,Massey,2018,65.168539,Alabama 38 Arizona 24 Arkans...


In [19]:
scores = pred_df.groupby(['Method','days_to_subtract','domain','range','direct_thres','spread_thres','weight_indirect'])['Predictability'].median().reset_index()
scores.sort_values(by='Predictability').set_index('Method').loc['Colley']

,days_to_subtract,domain,range,direct_thres,spread_thres,weight_indirect,Predictability
Method,,,,,,,
Colley,21,all,madness,0,0,0,64.044944
Colley,7,all,madness,0,0,0,64.285714
Colley,35,all,madness,0,0,0,64.285714
Colley,14,all,madness,0,0,0,64.444444
Colley,28,all,madness,0,0,0,64.444444


In [20]:
inx_values = ['domain','range','direct_thres','spread_thres','weight_indirect']
best_df = scores.groupby(['days_to_subtract','Method']).apply(lambda df: pd.Series(df.set_index(inx_values).idxmax()[0],index=inx_values)).reset_index()
best_df

,days_to_subtract,Method,domain,range,direct_thres,spread_thres,weight_indirect
0,7,Colley,all,madness,0,0,0
1,7,Massey,all,madness,0,0,0
2,14,Colley,all,madness,0,0,0
3,14,Massey,all,madness,0,0,0
4,21,Colley,all,madness,0,0,0
5,21,Massey,all,madness,0,0,0
6,28,Colley,all,madness,0,0,0
7,28,Massey,all,madness,0,0,0
8,35,Colley,all,madness,0,0,0
9,35,Massey,all,madness,0,0,0


In [21]:
import altair as alt

graph_df = best_df.set_index(inx_values+["days_to_subtract","Method"]).join(scores.set_index(inx_values+["days_to_subtract","Method"])).reset_index().sort_values(by='days_to_subtract')

g = alt.Chart(graph_df).mark_line().encode(
    x='days_to_subtract',
    y=alt.Y('Predictability',scale=alt.Scale(domain=[60, 70])),
    color='Method:N'
)

## How does predictability look?

In [22]:
g

alt.Chart(...)

In [23]:
best_pred_df = pred_df.set_index(inx_values+["days_to_subtract","Method"]).loc[best_df.set_index(inx_values+["days_to_subtract","Method"]).index]
best_pred_df

Year  \
domain range   direct_thres spread_thres weight_indirect days_to_subtract Method         
all    madness 0            0            0               7                Colley  2002   
                                                                          Colley  2003   
                                                                          Colley  2004   
                                                                          Colley  2005   
                                                                          Colley  2006   
...                                                                                ...   
                                                         35               Massey  2014   
                                                                          Massey  2015   
                                                                          Massey  2016   
                                                                          Massey  2017   
                                                                          Massey  2018   

                                                                                  Predictability  \
domain range   direct_thres spread_thres weight_indirect days_to_subtract Method                   
all    madness 0            0            0               7                Colley       64.044944   
                                                                          Colley       67.857143   
                                                                          Colley       65.591398   
                                                                          Colley       65.934066   
                                                                          Colley       60.465116   
...                                                                                          ...   
                                                         35               Massey       65.000000   
                                                                          Massey       69.387755   
                                                                          Massey       65.476190   
                                                                          Massey       54.838710   
                                                                          Massey       62.921348   

                                                                                                                           rankings  
domain range   direct_thres spread_thres weight_indirect days_to_subtract Method                                                     
all    madness 0            0            0               7                Colley  Alabama            6
Alcorn_St         62
Ariz...  
                                                                          Colley  Alabama        35
Arizona         1
Arizona_St...  
                                                                          Colley  Air_Force      40
Alabama        32
Alabama_St...  
                                                                          Colley  Alabama           17
Alabama_A&M       65
Ariz...  
                                                                          Colley  Air_Force        34
Alabama          44
Albany...  
...                                                                                                                             ...  
                                                         35               Massey  Albany_NY        65
American_Univ    57
Arizon...  
                                                                          Massey  Albany_NY     59
Arizona        8
Arkansas    ...  
                                                                          Massey  Akron              54
Arizona            14
Ar...  
                                                                          Massey  Arizona          26
Arkansas         38
Baylor...  
  

In [24]:
best_pred_df.reset_index().set_index('days_to_subtract').loc[14]

,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,Predictability,rankings
days_to_subtract,,,,,,,,,
14,all,madness,0,0,0,Colley,2002,66.292135,Alabama 6 Alcorn_St 62 Ariz...
14,all,madness,0,0,0,Colley,2003,65.476190,Alabama 34 Arizona 1 Arizona_St...
14,all,madness,0,0,0,Colley,2004,61.290323,Air_Force 41 Alabama 33 Alabama_St...
14,all,madness,0,0,0,Colley,2005,68.131868,Alabama 20 Alabama_A&M 64 Ariz...
14,all,madness,0,0,0,Colley,2006,61.627907,Air_Force 34 Alabama 36 Albany...
14,all,madness,0,0,0,Colley,2007,79.761905,Albany_NY 57 Arizona 15 Arkansas 4...
14,all,madness,0,0,0,Colley,2008,66.292135,American_Univ 59 Arizona 38 Arkans...
14,all,madness,0,0,0,Colley,2009,64.444444,Akron 56 Alabama_St 62 Americ...
14,all,madness,0,0,0,Colley,2010,64.444444,Ark_Pine_Bluff 65 BYU 15 Bayl...


In [28]:
top_k = 10
feature_names = [f'top{top_k}_intersection',f'top{top_k}_tau']
ms = pd.DataFrame(columns=['days_to_subtract1','days_to_subtract2','domain','range',"direct_thres","spread_thres","weight_indirect",'Method','Year']+feature_names)

#keys = list(itertools.product(domains_ranges,direct_thress,spread_thress,weight_indirects,years))
pair_days_to_subtracts = [sorted(days_to_subtracts) for days_to_subtracts in list(itertools.combinations(days_to_subtracts,2))]

c=0
for index,row in best_df.iterrows():
    dom,ran,dt,st,iw = row.loc['domain'],row.loc['range'],row.loc['direct_thres'],row.loc['spread_thres'],row.loc['weight_indirect']
    method = row.loc['Method']
    for year in years:
        for days_to_subtract1,days_to_subtract2 in pair_days_to_subtracts:
            if method == 'Massey':
                rankings = massey_rankings[(domain_range,year)].set_index(["direct_thres","spread_thres","weight_indirect"]).loc[(dt,st,iw)]
            elif method == 'Colley':
                rankings = colley_rankings[(domain_range,year)].set_index(["direct_thres","spread_thres","weight_indirect"]).loc[(dt,st,iw)]
                #import pdb; pdb.set_trace()
            else:
                raise Exception('Unsupported')
            rankings1 = rankings.set_index('days_to_subtract').loc[days_to_subtract1].T
            rankings1 = rankings1.loc[rankings1 < top_k]
            rankings2 = rankings.set_index('days_to_subtract').loc[days_to_subtract2].T
            rankings2_rankings1 = rankings2.loc[rankings1.index]
            rankings2 = rankings2.loc[rankings2 < top_k]
            val = len(set(rankings1.index).intersection(set(rankings2.index)))/len(set(rankings1.index).union(set(rankings2.index)))#top_k
            tau, p_value = stats.kendalltau(np.argsort(rankings1.values), np.argsort(rankings2.values))
            entry = pd.Series([days_to_subtract1,days_to_subtract2,dom,ran,dt,st,iw,method,year,val,tau],name=c,index=ms.columns)
            c+=1
            ms=ms.append(entry)

## What if we compare the top 15 teams before and after we add in the remaining games?
* So at 0.5 we have 50% of games still left remaining to add. 
* We can compute the intersection of the top 15 teams before and after the new games

In [29]:
ms.sort_values(by=feature_names[0])

,days_to_subtract1,days_to_subtract2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,top10_intersection,top10_tau
113,7,35,all,madness,0,0,0,Colley,2013,0.384615,0.166667
793,7,35,all,madness,0,0,0,Colley,2013,0.384615,0.166667
693,7,35,all,madness,0,0,0,Colley,2003,0.384615,0.055556
1373,7,35,all,madness,0,0,0,Colley,2003,0.384615,0.055556
1473,7,35,all,madness,0,0,0,Colley,2013,0.384615,0.166667
...,...,...,...,...,...,...,...,...,...,...,...
1354,21,28,all,madness,0,0,0,Massey,2018,1.000000,0.555556
666,7,28,all,madness,0,0,0,Massey,2017,1.000000,-0.388889
385,14,28,all,madness,0,0,0,Colley,2006,1.000000,0.611111
674,21,28,all,madness,0,0,0,Massey,2018,1.000000,0.555556


In [30]:
graph_df = ms.copy().reset_index()
graph_df["Interval"] = graph_df["days_to_subtract1"].astype(str) +"-"+graph_df["days_to_subtract2"].astype(str)
graph_df["Interval Width"] = -(graph_df["days_to_subtract1"] - graph_df["days_to_subtract2"])
graph_df[f"Intersection in top {top_k}"] = graph_df[feature_names[0]]
graph_df[f"Tau in top {top_k}"] = graph_df[feature_names[1]]

g = alt.Chart(graph_df).mark_line().encode(
    x='Year',
    y=alt.Y(f"average(Intersection in top {top_k})",scale=alt.Scale(domain=[0, 1.])),
    color='Interval Width:N'
).facet(column='Method')

## What does the average intersection look like?

In [31]:
g

alt.FacetChart(...)

In [33]:
alt.Chart(graph_df).mark_line().encode(
    x='Year',
    y=alt.Y(f"average(Tau in top {top_k})",scale=alt.Scale(domain=[-1, 1.])),
    color='Interval Width:N'
).facet(column='Method')

alt.FacetChart(...)

In [34]:
import altair as alt

graph_df = ms.copy().reset_index()
graph_df["Interval"] = graph_df["days_to_subtract1"].astype(str) +"-"+graph_df["days_to_subtract2"].astype(str)
graph_df["Interval Width"] = -(graph_df["days_to_subtract1"] - graph_df["days_to_subtract2"])
graph_df[f"Intersection in top {top_k}"] = graph_df[feature_name]

g = alt.Chart(graph_df).mark_bar().encode(
    y=f"average(Intersection in top {top_k})",
    x='Interval Width:N',
    color='Method:O',
    column='Method'
)

In [35]:
import altair as alt

graph_df = ms.copy().reset_index()
graph_df["Interval"] = graph_df["days_to_subtract1"].astype(str) +"-"+graph_df["days_to_subtract2"].astype(str)
graph_df["Interval Width"] = -(graph_df["days_to_subtract1"] - graph_df["days_to_subtract2"])
graph_df[f"Intersection in top {top_k}"] = graph_df[feature_name]

error_bars = alt.Chart(graph_df).mark_errorbar(extent='ci').encode(
    x='Interval Width:N',
    y=alt.Y(f"average(Intersection in top {top_k})",scale=alt.Scale(domain=[0.5, 1.])),
    color='Method'
)

g = error_bars+alt.Chart(graph_df).mark_line().encode(
    x='Interval Width:N',
    y=alt.Y(f"average(Intersection in top {top_k})",scale=alt.Scale(domain=[0.5, 1.])),
    color='Method'
)

## Here is a view grouped by the interval width

In [36]:
g

alt.LayerChart(...)

In [38]:
sensitivity_target = ms.copy()

sensitivity_data = {}
for year in tqdm(years):
    sensitivity_data[year] = {}
    for days_to_subtract in days_to_subtracts:
        game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                                "team1_score":games[year]['points1'],
                                "team1_H_A_N": games[year]['H_A_N1'],
                                "team2_name":games[year]['team2_name'],
                                "team2_score":games[year]['points2'],
                                "team2_H_A_N": games[year]['H_A_N1'],
                                "date": games[year]['date']
                               }).sort_values(by='date')
        mask = game_df.team1_name.isin(team_domain) & game_df.team2_name.isin(team_domain)
        game_df = game_df.loc[mask]
        delta = timedelta(days=days_to_subtract)
        game_df_sample = game_df.loc[game_df["date"] <= pd.to_datetime(base.selectionSundays[year],format="%m/%d/%Y")-delta].drop('date',axis=1)
        sensitivity_data[year][f"days_to_subtract={days_to_subtract}"]=game_df_sample
        
joblib.dump({'description':major_description,'target':sensitivity_target,'data':sensitivity_data,'other':{'madness_teams':madness_teams,'remaining_games':remaining_games,'best_df':best_df,'best_pred_df':best_pred_df,'top_k':top_k,'feature_names':feature_names}},"/disk/RPLib/problem_0001.joblib.z")

100%|██████████| 17/17 [00:00<00:00, 45.96it/s]


['/disk/RPLib/problem_0001.joblib.z']